In [1]:
library(DESeq2)
library(gplots)
library(RColorBrewer)
library(genefilter)
library(heatmap3)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Warning message:


In [2]:
countsFile <- '/counts/all_samples.countSimp.no_header.txt'
sampleInfoFile <- 'all_samples_extended_info.txt'
peakIDsFile <- 'dlpfc_hpc_combined_set_combined_200_name.saf'

In [3]:
countMatrix <- read.table(countsFile, header = FALSE, sep = "\t", skip = 0)
sampleInfo <- read.table(sampleInfoFile, header = TRUE, row.names=1, sep = "\t", skip=0)
peakInfo <- read.table(peakIDsFile, header = FALSE, row.names=1, sep = "\t", skip=0)

#making sure the row names are valid variable names, essential for row names and column names to match
rownames(sampleInfo) <- make.names(rownames(sampleInfo))
all(rownames(sampleInfo) == colnames(countMatrix))
colnames(countMatrix) <- rownames(sampleInfo)
all(rownames(sampleInfo) == colnames(countMatrix))

[1] FALSE

[1] TRUE

In [4]:
sampleInfo$proj_id <- factor(sampleInfo$proj_id)
sampleInfo$cell_type <- factor(sampleInfo$cell_type)
sampleInfo$brain_region <- factor(sampleInfo$brain_region)
sampleInfo$attempt <- factor(sampleInfo$attempt)
sampleInfo$replicate <- factor(sampleInfo$replicate)
sampleInfo$binary_amyloid <- factor(sampleInfo$binary_amyloid)
sampleInfo$msex <- factor(sampleInfo$msex)
sampleInfo$is_microglia <- factor(as.integer(sampleInfo$cell_type == "Microglia"))
sampleInfo$is_glia <- factor(as.integer(sampleInfo$cell_type == "Glia"))
sampleInfo$is_neuron <- factor(as.integer(sampleInfo$cell_type == "Neuron"))

In [6]:
countMatrix <- countMatrix[,rownames(sampleInfo)]

In [7]:
all(rownames(sampleInfo) == colnames(countMatrix))

[1] TRUE

In [8]:
dds <-  DESeqDataSetFromMatrix(countData = countMatrix,
                              colData = sampleInfo,
                              design = ~ binary_amyloid)
dds

class: DESeqDataSet 
dim: 352012 155 
metadata(1): version
assays(1): counts
rownames: NULL
rowData names(0):
colnames(155): X10290265_Neuron_HPC_1_1 X10290265_Neuron_HPC_1_2 ...
  X21142003_Neuron_DLPFC_1_1 X21142003_Neuron_DLPFC_1_2
colData names(22): proj_id cell_type ... is_glia is_neuron

In [9]:
ddsCollapsed <- collapseReplicates( dds,
                                   groupby = make.names(paste(dds$proj_id,dds$brain_region, dds$cell_type,sep="_")))

In [10]:
count_vst <- assay(vst(ddsCollapsed))

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.


In [13]:
rownames(count_vst) <- rownames(peakInfo)

In [14]:
asc1_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_ASC1_markers.txt", what=character())
asc2_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_ASC2_markers.txt", what=character())
end_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_END_markers.txt", what=character())
exca1_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_exCA1_markers.txt", what=character())
exca3_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_exCA3_markers.txt", what=character())
exdg_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_exDG_markers.txt", what=character())
expfc1_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_exPFC1_markers.txt", what=character())
expfc2_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_exPFC2_markers.txt", what=character())
gaba1_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_GABA1_markers.txt", what=character())
gaba2_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_GABA2_markers.txt", what=character())
mg_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_MG_markers.txt", what=character())
nsc_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_NSC_markers.txt", what=character())
odc1_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_ODC1_markers.txt", what=character())
odc2_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_ODC2_markers.txt", what=character())
opc_markers <- scan("/habib_markers_analysis/dlpfc_hpc_peaks_corresponding_to_habib_OPC_markers.txt", what=character())

In [15]:
sampleInfo <- colData(ddsCollapsed)
sampleInfoSorted <- sampleInfo[order(sampleInfo$cell_type, sampleInfo$brain_region),]

In [16]:
count_vst <- count_vst[,rownames(sampleInfoSorted)]

In [17]:
markers_list <- c(expfc1_markers,
                  expfc2_markers,
                  exca1_markers,
                  exca3_markers,
                  exdg_markers,
                  gaba1_markers,
                  gaba2_markers,
                  mg_markers,
                  odc1_markers,
                  odc2_markers,
                  opc_markers,
                  asc1_markers,
                  asc2_markers,
                  nsc_markers,
                  end_markers)

markers_colors <- c(rep("brown", length(expfc1_markers)),
                    rep("brown1", length(expfc2_markers)),
                    rep("brown2", length(exca1_markers)),
                    rep("brown3", length(exca3_markers)),
                    rep("coral", length(exdg_markers)),
                    rep("coral1", length(gaba1_markers)),
                    rep("coral2", length(gaba2_markers)),
                    rep("blue", length(mg_markers)),
                    rep("chocolate", length(odc1_markers)),
                    rep("chocolate1", length(odc2_markers)),
                    rep("chocolate4", length(opc_markers)),
                    rep("darkorchid", length(asc1_markers)),
                    rep("darkorchid4", length(asc2_markers)),
                    rep("cornsilk", length(nsc_markers)),
                    rep("darkgoldenrod", length(end_markers))
                    )

In [18]:
count_vst <- count_vst[markers_list,]

In [19]:
########### Gets breaks to use for heatmap #############
#num is how mnany values you want on each side of 0
getBreaks2 <- function(absMax,num){
    x <- c(-num:num)
    y <- x*(absMax/num);
    return(y);
}

curQuantV <- quantile(abs(count_vst),c(0:100)*.01)
curPlotBkV <- getBreaks2(curQuantV[95],20);
pdf("markers_heatmap.pdf")
heatmap3(count_vst,Colv=NA, Rowv=NA, scale="row",
                labRow="",
                labCol="",
                col=colorRampPalette(c("blue","white","red"))(length(curPlotBkV)-1),
                ColSideColors=topo.colors(3)[sampleInfoSorted$brain_region],
                RowSideColors=markers_colors,
                useRaster = F,
                breaks=curPlotBkV, mar=c(15,10), cexRow=1.2, cexCol=1.2, cex.axis=1.2
                );
dev.off()

png 
  2

In [20]:
write.table(count_vst, file="/habib_markers_analysis/habib_marker_peaks_vst_count_matrix.txt", sep="\t", row.names=TRUE, col.names=TRUE)
write.table(sampleInfoSorted, file="/habib_markers_analysis/habib_marker_peaks_sample_info_sorted.txt", sep="\t", row.names=TRUE, col.names=TRUE)

In [21]:
print(length(expfc1_markers))
print(length(expfc2_markers))
print(length(exca1_markers))
print(length(exca3_markers))
print(length(exdg_markers))
print(length(gaba1_markers))
print(length(gaba2_markers))
print(length(mg_markers))
print(length(odc1_markers))
print(length(odc2_markers))
print(length(opc_markers))
print(length(asc1_markers))
print(length(asc2_markers))
print(length(nsc_markers))
print(length(end_markers))

[1] 3389
[1] 1922
[1] 2887
[1] 4010
[1] 5542
[1] 3410
[1] 4262
[1] 2078
[1] 2900
[1] 4751
[1] 1457
[1] 2422
[1] 3172
[1] 5373
[1] 5573
